# Série 2 - Exercice 1

On va faire quelques requêtes SSL vers des sites pour voir comment ils répondent.

## 1. Connaissance

La façon la plus simple de faire ces connexions est en fait d'utiliser le navigateur. 
Autant Mozilla Firefox, Chrome, Safari, Edge, vous permettent de montrer les certificats
utilisés pour sécuriser le site visité.

Donc pour cette première partie, visitez les sites suivants avec votre navigateur, et trouvez:
- le nombre de certificats dans la liste
- le certificat de base

Les sites sont:
- https://google.ch
- https://google.com
- https://moodle.unifr.ch/
- https://web.fledg.re

Est-ce que vous trouvez le site qui utilise Let's Encrypt?

## 2. Compréhension

Utiliser le navigateur est simple, mais en utilisant Python on voit un peu plus comment ça marche.
Dans l'exemple en bas, il y a quelques méthods qui ont été créées, tout d'abord ceux-ci:

- ´get_connection_chain´ se connecte au serveur indiqué et retourne la connection et la chaîne de certificats - ATTENTION: il faut seulement donner le nom de domaine, pas l'URL complète!
- ´dump_cert´ imprime un certificat
- ´dump_chain´ imprime toute la chaîne

Qu'est-ce que vous observez en lançant le code? Pourquoi c'est différent?

Grâce au Python, nous avons aussi de l'information supplémentaire sur la connexion. Essayez les deux adresses suivantes:
- https://xkcd.com
- https://sssscomic.com


In [1]:
# Série 2 - Exercice 1 - Partie 2

from OpenSSL import SSL, crypto
import socket, certifi

def dump_cert(cert):
    for component in cert.get_subject().get_components():
        print("Subject %s: %s" % (component))
             
    print("notBefore:", cert.get_notBefore())
    print("notAfter:", cert.get_notAfter())
    print("version:" + str(cert.get_version()))
    print("sigAlg:", cert.get_signature_algorithm())
    print("digest:", cert.digest('sha256'))
    print("issuer:", cert.get_issuer())
    print()
    
def get_connection_chain(host, port = 443):
    dst = (str.encode(host), port)
    ctx = SSL.Context(SSL.SSLv23_METHOD)
    s = socket.create_connection(dst)
    s = SSL.Connection(ctx, s)
    s.set_connect_state()
    s.set_tlsext_host_name(dst[0])

    s.sendall(b'HEAD / HTTP/1.0\n\n')
    s.recv(16)
    return (s, s.get_peer_cert_chain())

def dump_chain(chain):
    for pos, cert in enumerate(chain):
        print("Certificate #" + str(pos))
        dump_cert(cert)
        
def dump_connection(conn):
    print("Cipher name:", conn.get_cipher_name())
    print("Cipher size:", conn.get_cipher_bits())
    print("Cipher version:", conn.get_cipher_version())
    print("Cipher list:", conn.get_cipher_list())

conn, chain = get_connection_chain("google.ch")
dump_chain(chain)
dump_connection(conn)

Certificate #0
Subject b'CN': b'*.google.ch'
notBefore: b'20211101033446Z'
notAfter: b'20220124033445Z'
version:2
sigAlg: b'sha256WithRSAEncryption'
digest: b'8F:0A:18:53:ED:50:00:5E:A5:71:14:88:E0:74:37:A8:B0:F2:0D:FD:76:0B:6B:08:CC:FA:AC:64:64:A9:AA:C0'
issuer: <X509Name object '/C=US/O=Google Trust Services LLC/CN=GTS CA 1C3'>

Certificate #1
Subject b'C': b'US'
Subject b'O': b'Google Trust Services LLC'
Subject b'CN': b'GTS CA 1C3'
notBefore: b'20200813000042Z'
notAfter: b'20270930000042Z'
version:2
sigAlg: b'sha256WithRSAEncryption'
digest: b'23:EC:B0:3E:EC:17:33:8C:4E:33:A6:B4:8A:41:DC:3C:DA:12:28:1B:BC:3F:F8:13:C0:58:9D:6C:C2:38:75:22'
issuer: <X509Name object '/C=US/O=Google Trust Services LLC/CN=GTS Root R1'>

Certificate #2
Subject b'C': b'US'
Subject b'O': b'Google Trust Services LLC'
Subject b'CN': b'GTS Root R1'
notBefore: b'20200619000042Z'
notAfter: b'20280128000042Z'
version:2
sigAlg: b'sha256WithRSAEncryption'
digest: b'3E:E0:27:8D:F7:1F:A3:C1:25:C4:CD:48:7F:01:D7:74:6

## 3. Application

Pour la troisième partie, on va enfin aller chercher le certificat de base. Comme on a vu dans la partie 2, le certificat de base n'est pas envoyé par le site. Ca donne du sens, parce qu'on ne fait à priori pas confiance au site. Donc on ne va pas accepter un certificat qui vient de là.

Il faut donc avoir une liste de certificats à qui on fait confiance. Cette liste réside sur votre ordinateur, et peut être récupérée avec ´certifi.where()´.

Vous pouvez maintenant utiliser la méthode ´get_root_cert´ sur un des certificats précédents pour vous retourner la chaîne de certificats de base.

Question:
- pourquoi on donne seulement le dernier élément de la liste (`chain[-1]`)?
- pourquoi on reçoit deux certificats à la fin?

In [2]:
# Série 2 - Exercice 1 - Partie 3

def get_root_chain(chain):
    store = crypto.X509Store()
    store.load_locations(certifi.where())
    store_ctx = crypto.X509StoreContext(store, chain[-2])
    return store_ctx.get_verified_chain()

dump_chain(get_root_chain(chain))


Certificate #0
Subject b'C': b'US'
Subject b'O': b'Google Trust Services LLC'
Subject b'CN': b'GTS CA 1C3'
notBefore: b'20200813000042Z'
notAfter: b'20270930000042Z'
version:2
sigAlg: b'sha256WithRSAEncryption'
digest: b'23:EC:B0:3E:EC:17:33:8C:4E:33:A6:B4:8A:41:DC:3C:DA:12:28:1B:BC:3F:F8:13:C0:58:9D:6C:C2:38:75:22'
issuer: <X509Name object '/C=US/O=Google Trust Services LLC/CN=GTS Root R1'>

Certificate #1
Subject b'C': b'US'
Subject b'O': b'Google Trust Services LLC'
Subject b'CN': b'GTS Root R1'
notBefore: b'20160622000000Z'
notAfter: b'20360622000000Z'
version:2
sigAlg: b'sha384WithRSAEncryption'
digest: b'2A:57:54:71:E3:13:40:BC:21:58:1C:BD:2C:F1:3E:15:84:63:20:3E:CE:94:BC:F9:D3:CC:19:6B:F0:9A:54:72'
issuer: <X509Name object '/C=US/O=Google Trust Services LLC/CN=GTS Root R1'>

